<img src="./img/logos_combined.png" align="right" width="50%"></img>
<br><br><br>

# es-data-lib - Example and use cases

<hr>
<a href="./01_owslib_example.ipynb"><< OWSlib example</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./03_es_data_lib.ipynb">es-data-lib example >></a>